In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전/입실렌티전야제

/content/drive/.shortcut-targets-by-id/11I-rKkyrnUaOhSTzVScvbROijR4qxGDQ/BDA 공모전/입실렌티전야제


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
emp = pd.read_csv('emp_train_v4.csv', index_col=0)
reg = pd.read_csv('reg_train_v4.csv', index_col=0)
# 뺄 것: product_name,employee_yn, real_product, matches
# dummy: gender, brand, age_gen, real_product, prod_processed, matches
# prime_yn은 0, 1로 바꾸기

In [ ]:
emp_x = emp.drop(['product_name', 'employee_yn','real_product','matches','prod_processed','prime_yn'], axis=1)
emp_x = pd.get_dummies(emp_x)
emp_y = emp['prime_yn']
for i in range(len(emp_y)):
  if emp_y[i] == 'Y':
    emp_y[i] = 1
  else:
    emp_y[i] = 0
emp_y = emp_y.astype(int)

<ipython-input-6-2fab08e26a04>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emp_y[i] = 0
<ipython-input-6-2fab08e26a04>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emp_y[i] = 1


In [ ]:
reg_x = reg.drop(['product_name', 'employee_yn','real_product','matches','prod_processed','prime_yn'], axis=1)
reg_x = pd.get_dummies(reg_x)
reg_y = reg['prime_yn']
for i in range(len(reg_y)):
  if reg_y[i] == 'Y':
    reg_y[i] = 1
  else:
    reg_y[i] = 0
reg_y = reg_y.astype(int)

<ipython-input-7-3a0bb4bf5e09>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_y[i] = 0
<ipython-input-7-3a0bb4bf5e09>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_y[i] = 1


## 변수 선택

### VIF
다중공선성 확인

emp: scd, order_date, mid_cat2, main_cat2, gender_F, gender_M, age_gen

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

emp_vif = pd.DataFrame()

lst = []

# 독립변수의 갯수 만큼 반복
for i in range(emp_x.shape[1]):
    v = variance_inflation_factor(emp_x.values, i) # 컬럼을 정수로 지정한다.
    lst.append(v)
    
emp_vif["VIF Factor"] = lst
emp_vif["features"] = emp_x.columns
emp_vif

,VIF Factor,features
0,9266.583457,scd
1,1.522078,net_order_qty
2,2.778861,net_order_amt
3,4.518240,age_grp
4,9184.593416,order_date
5,1.444239,total_order_amt
6,12.058968,festival
7,1.020359,salary
8,1.001080,limit
9,1.076943,pre_dcnt


reg: scd, order_date, mid_cat2, main_cat2, gender_F, gender_M, age_gen

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

reg_vif = pd.DataFrame()

lst = []

# 독립변수의 갯수 만큼 반복
for i in range(reg_x.shape[1]):
    v = variance_inflation_factor(reg_x.values, i) # 컬럼을 정수로 지정한다.
    lst.append(v)
    
reg_vif["VIF Factor"] = lst
reg_vif["features"] = reg_x.columns
reg_vif

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF Factor,features
0,26563.074173,scd
1,1.643719,net_order_qty
2,2.201763,net_order_amt
3,4.280550,age_grp
4,25252.539115,order_date
5,1.384006,total_order_amt
6,28.004574,festival
7,1.585781,salary
8,1.843960,limit
9,NaN,pre_dcnt


## logistic regression

emp: scd, order_date, mid_cat2, main_cat2, gender_F, gender_M, age_gen

### emp 데이터셋

In [ ]:
emp_x2 = emp_x.drop(['scd','order_date','main_cat2','gender_F','gender_M'], axis=1)
reg_x2 = reg_x.drop(['scd','order_date','main_cat2','gender_F','gender_M'], axis=1)
emp_x3 = emp_x.iloc[:,0:26]
reg_x3 = reg_x.iloc[:,0:24]

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
scaler = StandardScaler()
emp_x4 = pd.DataFrame(scaler.fit_transform(emp_x3))
reg_x4 = pd.DataFrame(scaler.fit_transform(reg_x3))

emp_x4.columns = emp_x3.columns
reg_x4.columns = reg_x3.columns

In [ ]:
emp_x5 = emp_x3.drop(['main_cat2','mid_cat2','scd'], axis=1)
reg_x5 = reg_x.iloc[:,0:25].drop(['main_cat2','mid_cat2','scd'], axis=1)

scaler = StandardScaler()
emp_x6 = pd.DataFrame(scaler.fit_transform(emp_x5))
reg_x6 = pd.DataFrame(scaler.fit_transform(reg_x5))

emp_x6.columns = emp_x5.columns
reg_x6.columns = reg_x5.columns

In [ ]:
# brand랑 age_gen만 포함된 X
emp_x7 = emp[['age_gen']]
reg_x7 = reg[['age_gen']]
emp_x8 = pd.get_dummies(emp_x7)
reg_x8 = pd.get_dummies(reg_x7)

In [ ]:
import statsmodels.api as sm
logit3 = sm.Logit(emp_y, emp_x6)
result = logit3.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.674796
         Iterations 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               prime_yn   No. Observations:                19805
Model:                          Logit   Df Residuals:                    19783
Method:                           MLE   Df Model:                           21
Date:                Wed, 31 May 2023   Pseudo R-squ.:                -0.02173
Time:                        07:50:33   Log-Likelihood:                -13364.
converged:                       True   LL-Null:                       -13080.
Covariance Type:            nonrobust   LLR p-value:                     1.000
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
net_order_qty      -0.0009      0.020     -0.047      0.962      -0.039       0.037
net_order_amt      -0.0497      0.021     -2.321      0.020      -0.092      -0.008
age_grp             0.2466      0.015     16.746      0.000       0.218       0.275
order_date          0.0134      0.026      0.517      0.605      -0.037       0.064
total_order_amt     0.2803      0.019     14.573      0.000       0.243       0.318
festival           -0.0240      0.025     -0.953      0.341      -0.074       0.025
salary              0.0400      0.015      2.752      0.006       0.012       0.068
limit               0.0146      0.021      0.686      0.493      -0.027       0.056
pre_dcnt            0.0050      0.014      0.344      0.731      -0.023       0.033
colab              -0.0260      0.015     -1.766      0.077      -0.055       0.003
new_years           0.1040      0.020      5.304      0.000       0.066       0.142
early_del           0.0357      0.015      2.454      0.014       0.007       0.064
exclusive           0.0145      0.015      0.985      0.325      -0.014       0.043
dcnt                0.0498      0.015      3.369      0.001       0.021       0.079
box                -0.0124      0.015     -0.806      0.420      -0.043       0.018
expir              -0.0306      0.015     -2.044      0.041      -0.060      -0.001
set                 0.0238      0.015      1.613      0.107      -0.005       0.053
event               0.0523      0.015      3.541      0.000       0.023       0.081
capacity           -0.0361      0.018     -2.014      0.044      -0.071      -0.001
weight              0.0257      0.018      1.433      0.152      -0.009       0.061
gift                0.0216      0.016      1.382      0.167      -0.009       0.052
gender_F           -0.0471   9.54e+05  -4.94e-08      1.000   -1.87e+06    1.87e+06
gender_M            0.0471   9.54e+05   4.94e-08      1.000   -1.87e+06    1.87e+06
===================================================================================
"""

In [ ]:
import statsmodels.api as sm
logit4 = sm.Logit(emp_y, emp_x8)
result = logit4.fit()
result.summary()

         Current function value: 0.651545
         Iterations: 35


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               prime_yn   No. Observations:                19805
Model:                          Logit   Df Residuals:                    19799
Method:                           MLE   Df Model:                            5
Date:                Wed, 31 May 2023   Pseudo R-squ.:                 0.01348
Time:                        12:09:41   Log-Likelihood:                -12904.
converged:                      False   LL-Null:                       -13080.
Covariance Type:            nonrobust   LLR p-value:                 4.761e-74
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
age_gen_10 FM    -27.2316   2.73e+05  -9.98e-05      1.000   -5.35e+05    5.35e+05
age_gen_2030 F     0.1329      0.029      4.619      0.000       0.077       0.189
age_gen_2030 M     0.4623      0.026     17.547      0.000       0.411       0.514
age_gen_4050 F     0.7470      0.038     19.920      0.000       0.674       0.821
age_gen_4050 M     0.8105      0.030     27.392      0.000       0.753       0.869
age_gen_60 FM      1.0506      0.145      7.234      0.000       0.766       1.335
==================================================================================
"""

### reg 데이터셋

In [ ]:
reg_x6.drop(['pre_dcnt'], axis=1, inplace=True)

In [ ]:
import statsmodels.api as sm
logit4 = sm.Logit(reg_y, reg_x6)
result = logit4.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.653737
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               prime_yn   No. Observations:                26070
Model:                          Logit   Df Residuals:                    26050
Method:                           MLE   Df Model:                           19
Date:                Wed, 31 May 2023   Pseudo R-squ.:                 0.05427
Time:                        07:52:06   Log-Likelihood:                -17043.
converged:                       True   LL-Null:                       -18021.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
net_order_qty       0.0032      0.023      0.140      0.889      -0.042       0.049
net_order_amt      -0.2866      0.019    -15.260      0.000      -0.323      -0.250
age_grp             0.2843      0.013     21.330      0.000       0.258       0.310
order_date          0.1399      0.033      4.205      0.000       0.075       0.205
total_order_amt     0.4375      0.018     23.690      0.000       0.401       0.474
festival            0.1554      0.029      5.289      0.000       0.098       0.213
salary             -0.0856      0.016     -5.245      0.000      -0.118      -0.054
limit               0.1880      0.018     10.311      0.000       0.152       0.224
colab              -0.0234      0.014     -1.627      0.104      -0.052       0.005
new_years           0.1050      0.015      6.849      0.000       0.075       0.135
early_del           0.0061      0.013      0.462      0.644      -0.020       0.032
exclusive          19.9212      2.086      9.552      0.000      15.834      24.009
dcnt                0.0670      0.014      4.823      0.000       0.040       0.094
box                 0.0836      0.014      5.915      0.000       0.056       0.111
expir               0.0115      0.013      0.879      0.379      -0.014       0.037
set                 0.0226      0.013      1.684      0.092      -0.004       0.049
event              -0.0325      0.021     -1.556      0.120      -0.073       0.008
capacity            0.0014      0.017      0.083      0.934      -0.031       0.034
weight              0.0064      0.017      0.370      0.711      -0.027       0.040
gender_F           -0.1191   1.18e+06  -1.01e-07      1.000    -2.3e+06     2.3e+06
gender_M            0.1191   1.18e+06   1.01e-07      1.000    -2.3e+06     2.3e+06
===================================================================================
"""

In [ ]:
import statsmodels.api as sm
logit5 = sm.Logit(reg_y, reg_x8)
result = logit5.fit()
result.summary()

         Current function value: 0.679957
         Iterations: 35


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               prime_yn   No. Observations:                26070
Model:                          Logit   Df Residuals:                    26064
Method:                           MLE   Df Model:                            5
Date:                Wed, 31 May 2023   Pseudo R-squ.:                 0.01634
Time:                        12:09:54   Log-Likelihood:                -17726.
converged:                      False   LL-Null:                       -18021.
Covariance Type:            nonrobust   LLR p-value:                4.861e-125
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
age_gen_10 FM    -22.8776   1.79e+04     -0.001      0.999    -3.5e+04     3.5e+04
age_gen_2030 F    -0.6244      0.030    -20.732      0.000      -0.683      -0.565
age_gen_2030 M    -0.1543      0.041     -3.801      0.000      -0.234      -0.075
age_gen_4050 F    -0.1135      0.017     -6.585      0.000      -0.147      -0.080
age_gen_4050 M     0.3005      0.031      9.831      0.000       0.241       0.360
age_gen_60 FM      0.4586      0.070      6.521      0.000       0.321       0.596
==================================================================================
"""